In [ ]:
import tensorflow as tf

In [ ]:
from tensorflow.keras.layers import Conv2D, Flatten, Dense, MaxPool2D, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
import joblib
import PIL 
from PIL import Image
import h5py

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
tf.keras.backend.set_image_data_format('channels_first')

In [ ]:
def plot_metric(history, metric):
    train_metrics = history.history[metric]
    val_metrics = history.history['val_'+metric]
    epochs = range(1, len(train_metrics) + 1)
    plt.plot(epochs, train_metrics)
    plt.plot(epochs, val_metrics)
    plt.title('Training and validation '+ metric)
    plt.xlabel("Epochs")
    plt.ylabel(metric)
    plt.legend(["train_"+metric, 'val_'+metric])
    plt.show()

# Style

## Blazer

In [ ]:
img_height, img_width = (512, 512)
batch_size = 16

train_data_dir = r'//DRIVE/Algemeen/Interns/Mirle Willems/CLASSIFIER DATA/Style/Style - Blazer/processed_data/train'
valid_data_dir = r'//DRIVE/Algemeen/Interns/Mirle Willems/CLASSIFIER DATA/Style/Style - Blazer/processed_data/val'
test_data_dir = r'//DRIVE/Algemeen/Interns/Mirle Willems/CLASSIFIER DATA/Style/Style - Blazer/processed_data/test'

In [ ]:
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip = True) #validation_split=0.4

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')

valid_generator = train_datagen.flow_from_directory(
    valid_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')

In [ ]:
test_generator = train_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_height, img_width),
    batch_size = 1,
    class_mode = 'binary')

In [ ]:
x,y=test_generator.next()
print(x.shape)
print(y.shape)

train_generator.num_classes

print(train_generator.classes)
print(train_generator.class_indices)

In [ ]:
#2028 --> No
# 829 --> Yes
#2857--> Total

#Weights for No:
print(2857/(2*2028))
#Weights for Yes:
print(2857/(2*829))

### Learning default with early stopping

In [ ]:
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

In [ ]:
base_model = ResNet50(include_top=False, weights='imagenet')
x = base_model.output
x = GlobalAveragePooling2D()(x) #ty out with multiple Dense layers and other layers added to the end
x = Dense(1024, activation='relu')(x)
predictions = Dense(train_generator.num_classes, activation='softmax')(x) #softmax in original categorical classification
model_blazer_1 = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False #=True takes more time, but if you keep them trainable (the pretrained weights)
    
model_blazer_1.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics = ['accuracy'])

history_blazer_1 = model_blazer_1.fit_generator(train_generator, epochs = 40, callbacks=[callback], validation_data = valid_generator, class_weight = {0: 0.70, 1: 1.72})

In [ ]:
test_loss, test_acc = model_blazer_1.evaluate_generator(test_generator, verbose=2)
print('\nTest accuracy:', test_acc)

In [ ]:
plot_metric(history_blazer_1, 'loss')

In [ ]:
model_blazer_1.save(r'//DRIVE/Algemeen/Interns/Mirle Willems/CLASSIFIER DATA/Style/Style - Blazer/saved_model/ResNet_Blazer_1.h5')

### Learning 0.0001 with early stopping

In [ ]:
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

In [ ]:
base_model = ResNet50(include_top=False, weights='imagenet')
x = base_model.output
x = GlobalAveragePooling2D()(x) #ty out with multiple Dense layers and other layers added to the end
x = Dense(1024, activation='relu')(x)
predictions = Dense(train_generator.num_classes, activation='softmax')(x) #softmax in original categorical classification
model_blazer_2 = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False #=True takes more time, but if you keep them trainable (the pretrained weights)

optimizer = Adam(lr=0.0001)

model_blazer_2.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics = ['accuracy'])

history_blazer_2 = model_blazer_2.fit_generator(train_generator, epochs = 40, callbacks=[callback], validation_data = valid_generator, class_weight ={0: 0.70, 1: 1.72})

In [ ]:
test_loss, test_acc = model_blazer_2.evaluate_generator(test_generator, verbose=2)
print('\nTest accuracy:', test_acc)

In [ ]:
plot_metric(history_blazer_2, 'loss')

In [ ]:
model_blazer_2.save(r'//DRIVE/Algemeen/Interns/Mirle Willems/CLASSIFIER DATA/Style/Style - Blazer/saved_model/ResNet_Blazer_2.h5')

### Learning 0.00001 with early stopping

In [ ]:
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

In [ ]:
base_model = ResNet50(include_top=False, weights='imagenet')
x = base_model.output
x = GlobalAveragePooling2D()(x) #ty out with multiple Dense layers and other layers added to the end
x = Dense(1024, activation='relu')(x)
predictions = Dense(train_generator.num_classes, activation='softmax')(x) #softmax in original categorical classification
model_blazer_3 = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False #=True takes more time, but if you keep them trainable (the pretrained weights)

optimizer = Adam(lr=0.00001)

model_blazer_3.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics = ['accuracy'])

history_blazer_3 = model_blazer_3.fit_generator(train_generator, epochs = 40, callbacks=[callback], validation_data = valid_generator, class_weight =  {0: 0.70, 1: 1.72})

In [ ]:
test_loss, test_acc = model_blazer_3.evaluate_generator(test_generator, verbose=2)
print('\nTest accuracy:', test_acc)

In [ ]:
plot_metric(history_blazer_3, 'loss')

In [ ]:
model_blazer_3.save(r'//DRIVE/Algemeen/Interns/Mirle Willems/CLASSIFIER DATA/Style/Style - Blazer/saved_model/ResNet_Blazer_3.h5')

# Matrices

In [ ]:
import pandas as pd
import seaborn as sn
import tensorflow as tf

model = tf.keras.models.load_model('//DRIVE/Algemeen/Interns/Mirle Willems/CLASSIFIER DATA/Style/Style - Blazer/saved_model/ResNet_Blazer_1.h5')
nb_samples = len(test_generator)
y_prob=[]
y_act=[]
test_generator.reset()

for _ in range(nb_samples):
    X_test, Y_test = test_generator.next()
    y_prob.append(model.predict(X_test))
    y_act.append(Y_test)
    
predicted_class = [list(train_generator.class_indices.keys())[i.argmax()] for i in y_prob] 
actual_class = [list(train_generator.class_indices.keys())[int(i)] for i in y_act] 

out_df = pd.DataFrame(np.vstack([predicted_class, actual_class]).T,columns=['predicted_class', 'actual_class'])
confusion_matrix = pd.crosstab(out_df['actual_class'], out_df["predicted_class"], rownames=["Actual"], colnames=['Predicted'])

sn.heatmap(confusion_matrix, cmap='Blues', annot=True, fmt='d')
plt.show()

print('test accuracy : {}'.format((np.diagonal(confusion_matrix).sum()/confusion_matrix.sum().sum()*100)))

In [ ]:
import pandas as pd
import seaborn as sn
import tensorflow as tf

model = tf.keras.models.load_model('//DRIVE/Algemeen/Interns/Mirle Willems/CLASSIFIER DATA/Style/Style - Blazer/saved_model/ResNet_Blazer_2.h5')
y_prob=[]
y_act=[]
test_generator.reset()

for _ in range(nb_samples):
    X_test, Y_test = test_generator.next()
    y_prob.append(model.predict(X_test))
    y_act.append(Y_test)
    
predicted_class = [list(train_generator.class_indices.keys())[i.argmax()] for i in y_prob] 
actual_class = [list(train_generator.class_indices.keys())[int(i)] for i in y_act] 

out_df = pd.DataFrame(np.vstack([predicted_class, actual_class]).T,columns=['predicted_class', 'actual_class'])
confusion_matrix = pd.crosstab(out_df['actual_class'], out_df["predicted_class"], rownames=["Actual"], colnames=['Predicted'])

sn.heatmap(confusion_matrix, cmap='Blues', annot=True, fmt='d')
plt.show()

print('test accuracy : {}'.format((np.diagonal(confusion_matrix).sum()/confusion_matrix.sum().sum()*100)))

In [ ]:
import pandas as pd
import seaborn as sn
import tensorflow as tf

model = tf.keras.models.load_model('//DRIVE/Algemeen/Interns/Mirle Willems/CLASSIFIER DATA/Style/Style - Blazer/saved_model/ResNet_Blazer_3.h5')
y_prob=[]
y_act=[]
nb_samples = len(test_generator)
test_generator.reset()

for _ in range(nb_samples):
    X_test, Y_test = test_generator.next()
    y_prob.append(model.predict(X_test))
    y_act.append(Y_test)
    
predicted_class = [list(train_generator.class_indices.keys())[i.argmax()] for i in y_prob] 
actual_class = [list(train_generator.class_indices.keys())[int(i)] for i in y_act] 

out_df = pd.DataFrame(np.vstack([predicted_class, actual_class]).T,columns=['predicted_class', 'actual_class'])
confusion_matrix = pd.crosstab(out_df['actual_class'], out_df["predicted_class"], rownames=["Actual"], colnames=['Predicted'])

sn.heatmap(confusion_matrix, cmap='Blues', annot=True, fmt='d')
plt.show()

print('test accuracy : {}'.format((np.diagonal(confusion_matrix).sum()/confusion_matrix.sum().sum()*100)))

## Extra: 0.0001 with early stopping(5)

In [ ]:
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

In [ ]:
base_model = ResNet50(include_top=False, weights='imagenet')
x = base_model.output
x = GlobalAveragePooling2D()(x) #ty out with multiple Dense layers and other layers added to the end
x = Dense(1024, activation='relu')(x)
predictions = Dense(train_generator.num_classes, activation='softmax')(x) #softmax in original categorical classification
model_blazer_4 = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False #=True takes more time, but if you keep them trainable (the pretrained weights)

optimizer = Adam(lr=0.0001)

model_blazer_4.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics = ['accuracy'])

history_blazer_4 = model_blazer_4.fit_generator(train_generator, epochs = 80, callbacks=[callback], validation_data = valid_generator, class_weight ={0: 0.70, 1: 1.72})

In [ ]:
test_loss, test_acc = model_blazer_4.evaluate_generator(test_generator, verbose=2)
print('\nTest accuracy:', test_acc)

In [ ]:
plot_metric(history_blazer_4, 'loss')

In [ ]:
model_blazer_4.save(r'//DRIVE/Algemeen/Interns/Mirle Willems/CLASSIFIER DATA/Style/Style - Blazer/saved_model/ResNet_Blazer_4.h5')

In [ ]:
import pandas as pd
import seaborn as sn
import tensorflow as tf

model = tf.keras.models.load_model('//DRIVE/Algemeen/Interns/Mirle Willems/CLASSIFIER DATA/Style/Style - Blazer/saved_model/ResNet_Blazer_4.h5')
y_prob=[]
y_act=[]
nb_samples = len(test_generator)
test_generator.reset()

for _ in range(nb_samples):
    X_test, Y_test = test_generator.next()
    y_prob.append(model.predict(X_test))
    y_act.append(Y_test)
    
predicted_class = [list(train_generator.class_indices.keys())[i.argmax()] for i in y_prob] 
actual_class = [list(train_generator.class_indices.keys())[int(i)] for i in y_act] 

out_df = pd.DataFrame(np.vstack([predicted_class, actual_class]).T,columns=['predicted_class', 'actual_class'])
confusion_matrix = pd.crosstab(out_df['actual_class'], out_df["predicted_class"], rownames=["Actual"], colnames=['Predicted'])

sn.heatmap(confusion_matrix, cmap='Blues', annot=True, fmt='d')
plt.show()

print('test accuracy : {}'.format((np.diagonal(confusion_matrix).sum()/confusion_matrix.sum().sum()*100)))

## 0.00001 early stopping (5)

In [ ]:
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

In [ ]:
base_model = ResNet50(include_top=False, weights='imagenet')
x = base_model.output
x = GlobalAveragePooling2D()(x) #ty out with multiple Dense layers and other layers added to the end
x = Dense(1024, activation='relu')(x)
predictions = Dense(train_generator.num_classes, activation='softmax')(x) #softmax in original categorical classification
model_blazer_5 = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False #=True takes more time, but if you keep them trainable (the pretrained weights)

optimizer = Adam(lr=0.00001)

model_blazer_5.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics = ['accuracy'])

history_blazer_5 = model_blazer_5.fit_generator(train_generator, epochs = 80, callbacks=[callback], validation_data = valid_generator, class_weight = {0: 0.70, 1: 1.72})

In [ ]:
test_loss, test_acc = model_blazer_5.evaluate_generator(test_generator, verbose=2)
print('\nTest accuracy:', test_acc)

In [ ]:
plot_metric(history_blazer_5, 'loss')

In [ ]:
model_blazer_5.save(r'//DRIVE/Algemeen/Interns/Mirle Willems/CLASSIFIER DATA/Style/Style - Blazer/saved_model/ResNet_Blazer_5.h5')

In [ ]:
import pandas as pd
import seaborn as sn
import tensorflow as tf

model = tf.keras.models.load_model('//DRIVE/Algemeen/Interns/Mirle Willems/CLASSIFIER DATA/Style/Style - Blazer/saved_model/ResNet_Blazer_5.h5')
y_prob=[]
y_act=[]
nb_samples = len(test_generator)
test_generator.reset()

for _ in range(nb_samples):
    X_test, Y_test = test_generator.next()
    y_prob.append(model.predict(X_test))
    y_act.append(Y_test)
    
predicted_class = [list(train_generator.class_indices.keys())[i.argmax()] for i in y_prob] 
actual_class = [list(train_generator.class_indices.keys())[int(i)] for i in y_act] 

out_df = pd.DataFrame(np.vstack([predicted_class, actual_class]).T,columns=['predicted_class', 'actual_class'])
confusion_matrix = pd.crosstab(out_df['actual_class'], out_df["predicted_class"], rownames=["Actual"], colnames=['Predicted'])

sn.heatmap(confusion_matrix, cmap='Blues', annot=True, fmt='d')
plt.show()

print('test accuracy : {}'.format((np.diagonal(confusion_matrix).sum()/confusion_matrix.sum().sum()*100)))